In [ ]:
import anndata as ad
import pandas as pd
import scanpy as sc
import seaborn as sns
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import matplotlib.patches as patches
from collections import defaultdict
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.stats import pearsonr
from sklearn.metrics import pairwise_distances, silhouette_score, adjusted_rand_score, normalized_mutual_info_score
import leidenalg
import igraph as ig
import itertools
import plotly.graph_objects as go
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import matplotlib.patches as mpatches
from matplotlib import colors as mcolors
from matplotlib.patches import Patch

In [ ]:
#NOT full feature (only comparing between datasets)
#bipartite matrix

#RCC = pd.read_csv("/home/larissa/Documents/Masterarbeit/RCC_results/liana_log_norm/no_subset_glom_removed/T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("/home/larissa/Documents/Masterarbeit/MF_results/liana/all_celltypes/MF_lr_ready.csv", index_col=0)

#RCC = pd.read_csv("D:\\studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("D:\\studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col=0)

RCC = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col = 0)
MF = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col = 0)

RCC["lr_pair"] = RCC["gene_A"] + "_" + RCC["gene_B"]
MF["lr_pair"] = MF["gene_A"] + "_" + MF["gene_B"]

RCC["cluster_pair"] = RCC["source"] + "@" + RCC["target"]
MF["cluster_pair"] = MF["source"] + "@" + MF["target"]

RCC["lr_means_normed"] = (RCC["lr_means"]-RCC["lr_means"].min())/(RCC["lr_means"].max()-RCC["lr_means"].min())
MF["lr_means_normed"] = (MF["lr_means"]-MF["lr_means"].min())/(MF["lr_means"].max()-MF["lr_means"].min())

#lr_threshold
RCC_filtered = RCC[(RCC["lr_means"] >= 0.0) | (RCC["lr_means"] <=  -abs(0.0))]
MF_filtered = MF[(MF["lr_means"] >= 0.0) | (MF["lr_means"] <= -abs(0.0))]

RCC_grouped = (RCC_filtered.groupby("cluster_pair")
               .apply(lambda df: dict(zip(df["lr_pair"], df["lr_means_normed"])))
               .to_dict())

MF_grouped = (MF_filtered.groupby("cluster_pair")
              .apply(lambda df: dict(zip(df["lr_pair"], df["lr_means_normed"])))
              .to_dict())

# remove cell pairs that only have 5 or less unique LR pairs 
RCC_grouped = {k: v for k, v in RCC_grouped.items() if len(v) >= 5}
MF_grouped = {k: v for k, v in MF_grouped.items() if len(v) >= 5}

def weighted_jaccard_dict(dict1, dict2):
    all_keys = set(dict1.keys()).union(dict2.keys())
    vec1 = np.array([dict1.get(i, 0.0) for i in all_keys])
    vec2 = np.array([dict2.get(i, 0.0) for i in all_keys])
    return np.minimum(vec1, vec2).sum() / np.maximum(vec1, vec2).sum() if np.maximum(vec1, vec2).sum() != 0 else 0.0

# pairwise similarity
results = []
for MF_cluster, MF_dict in MF_grouped.items():
    for RCC_cluster, RCC_dict in RCC_grouped.items():
        score = weighted_jaccard_dict(MF_dict, RCC_dict)
        results.append((MF_cluster, RCC_cluster, score))

jaccard_df_comp = pd.DataFrame(results, columns=["MF_cluster", "RCC_cluster", "weighted_jaccard"])
#jaccard_df.to_csv("jaccard_score_weighted_cluster_pairs_thresholded.csv", index=False)

In [ ]:
#Number of cellpairs with this number of unique LR pairs

RCC_filtered.groupby("cluster_pair")["lr_pair"].size().hist(bins=200, grid = False)
#plt.xticks(ticks=[0, 10, 20,  30, 40, 50, 60, 70, 80, 85, 90, 95, 100],)
plt.xlabel("Amount of unique LR pairs in one cellpair []")
plt.ylabel("Number of cellpairs with this number of unique LR pairs []")
plt.show()

In [ ]:
#Number of cellpairs with this number of unique LR pairs small

RCC_filtered.groupby("cluster_pair")["lr_pair"].size().hist(bins=200, grid = False)
plt.xlim(0, 25)
plt.xlabel("Amount of unique LR pairs in one cellpair []")
plt.ylabel("Number of cellpairs with this number of unique LR pairs []")
plt.show()

In [ ]:
print((RCC_filtered.groupby("cluster_pair")["lr_pair"].size() < 5).sum())

In [ ]:
#Number of cellpairs with this number of unique LR pairs

MF_filtered.groupby("cluster_pair")["lr_pair"].size().hist(bins=200, grid = False)
#plt.xticks(ticks=[0, 10, 20,  30, 40, 50, 60, 70, 80, 85, 90, 95, 100],)
plt.xlabel("Amount of unique LR pairs in one cellpair []")
plt.ylabel("Number of cellpairs with this number of unique LR pairs []")
plt.show()


In [ ]:
#Number of cellpairs with this number of unique LR pairs small

MF_filtered.groupby("cluster_pair")["lr_pair"].size().hist(bins=200, grid = False)
plt.xlim(0, 25)
plt.xlabel("Amount of unique LR pairs in one cellpair")
plt.ylabel("Number of cellpairs with this number of unique LR pairs")
plt.show()

In [ ]:
(MF_filtered.groupby("cluster_pair")["lr_pair"].size() < 5).sum()

In [ ]:
#jaccard similarity distribution

jaccard_df_comp_dist = 1 - jaccard_df_comp["weighted_jaccard"]
ax = sns.violinplot(jaccard_df_comp, inner= "quart")
quant = np.quantile(jaccard_df_comp["weighted_jaccard"], 0.75)
print(quant)
jaccard_df_quant = jaccard_df_comp[jaccard_df_comp["weighted_jaccard"] > quant]
ax.set_xticklabels([])
plt.xlabel("Weighted Jaccard Similarity")

In [ ]:
#bipartite matrix heatmap

heatrmap_df = jaccard_df_comp.pivot(
    index="RCC_cluster", columns="MF_cluster", values="weighted_jaccard"
)

heatrmap_df = 1- heatrmap_df
clust = sns.clustermap(heatrmap_df, figsize=(30, 30))


In [ ]:
#cluster hierarchically

heatrmap_df = jaccard_df_comp.pivot(index="RCC_cluster", columns="MF_cluster", values="weighted_jaccard")
heatrmap_df = 1 - heatrmap_df

g = sns.clustermap(heatrmap_df, method='average', figsize = (30,30), yticklabels=0, xticklabels=0, 
                  cbar_pos=(0.12, 0.82, 0.03, 0.15))

g.ax_heatmap.set_xlabel("RCC cluster", fontsize=30)
g.ax_heatmap.set_ylabel("MF cluster", fontsize=30)
cbar = g.ax_heatmap.collections[0].colorbar
cbar.set_label("Jaccard distance", fontsize=30)
cbar.ax.tick_params(labelsize=25)    
cbar.ax.yaxis.set_label_position("left")

row_order = g.dendrogram_row.reordered_ind
col_order = g.dendrogram_col.reordered_ind
reordered_rows = [heatrmap_df.index[i] for i in row_order]
reordered_cols = [heatrmap_df.columns[i] for i in col_order]


#define dendrogram clustering threshold
row_clusters = fcluster(g.dendrogram_row.linkage, t=11, criterion='maxclust')
col_clusters = fcluster(g.dendrogram_col.linkage, t=12, criterion='maxclust')

# map to reordered labels
row_cluster_labels = pd.Series(row_clusters, index=heatrmap_df.index).loc[reordered_rows]
col_cluster_labels = pd.Series(col_clusters, index=heatrmap_df.columns).loc[reordered_cols]

ax = g.ax_heatmap

# find start and length of each cluster in order
def get_cluster_spans(cluster_labels):
    spans = []
    prev_label = None
    start_idx = 0
    for idx, label in enumerate(cluster_labels):
        if label != prev_label and prev_label is not None:
            spans.append((prev_label, start_idx, idx))
            start_idx = idx
        prev_label = label
    spans.append((prev_label, start_idx, len(cluster_labels)))
    return spans

def enforce_min_spacing(positions, min_dist):
    adjusted = [positions[0]]
    for p in positions[1:]:
        if p - adjusted[-1] < min_dist:
            p = adjusted[-1] + min_dist
        adjusted.append(p)
    return adjusted


row_spans = get_cluster_spans(row_cluster_labels.tolist())
row_centers = [(start + (end - start) / 2 - 0.5) for (_, start, end) in row_spans]
row_centers_adj = enforce_min_spacing(row_centers, min_dist=13)


# row cluster spans (y-axis)

for (label, start, end), y_adj in zip(row_spans, row_centers_adj):
    height = end - start
    rect = patches.Rectangle(
        xy=(-0.5, start - 0.5),
        width=len(col_cluster_labels),
        height=height,
        fill=False,
        edgecolor='blue',
        linewidth=2
    )
    ax.add_patch(rect)
    ax.text(
        x=-1.5,
        #y=start + height / 2 - 0.5,
        y=y_adj,
        s=f"RCC{label}",
        va='center',
        ha='right',
        fontsize=25,
        color='blue'
    )


# col cluster spans(x axis)
col_spans = get_cluster_spans(col_cluster_labels.tolist())
num_cols = len(col_cluster_labels)

# normalized centers 
col_centers = [(start + (end - start)/2) / num_cols for (_, start, end) in col_spans]

# enforce minimum spacing
def enforce_min_spacing_axes(positions, min_dist=1):
    adjusted = [positions[0]]
    for p in positions[1:]:
        if p - adjusted[-1] < min_dist:
            p = adjusted[-1] + min_dist
        adjusted.append(p)
    return adjusted

col_centers_adj = enforce_min_spacing_axes(col_centers, min_dist=0.03)

for (label, start, end), x_adj in zip(col_spans, col_centers_adj):
    # rectangle in data coordinates
    rect = patches.Rectangle(
        xy=(start - 0.5, -0.5),
        width=end - start,
        height=len(row_cluster_labels),
        fill=False,
        edgecolor='green',
        linewidth=2
    )
    ax.add_patch(rect)

    # label in axes coordinates
    ax.text(
        x=x_adj,
        y= 1.02,               
        transform=ax.transAxes,
        s=f"MF{label}",
        ha='center',
        va='top',
        fontsize=25,
        color='green',
        rotation=0
    )


In [ ]:
row_cluster_labels_df = pd.DataFrame(row_cluster_labels)
row_cluster_labels_df['RCC_cluster'] = row_cluster_labels_df.index
row_cluster_labels_df = pd.DataFrame(row_cluster_labels).rename(columns={0: "row_cluster"})

col_cluster_labels_df = pd.DataFrame(col_cluster_labels)
col_cluster_labels_df['MF_cluster'] = col_cluster_labels_df.index
col_cluster_labels_df = pd.DataFrame(col_cluster_labels).rename(columns={0: "col_cluster"})

jaccard_df_compp = jaccard_df_comp.merge(row_cluster_labels_df, on="RCC_cluster")
jaccard_df_compp = jaccard_df_compp.merge(col_cluster_labels_df, on="MF_cluster")


In [ ]:
df_heatmap = (
    jaccard_df_compp
    .groupby(["row_cluster", "col_cluster"])["weighted_jaccard"]
    .median()
    .reset_index()
    .pivot(index="row_cluster", columns="col_cluster", values="weighted_jaccard")
)
df_heatmap = 1- df_heatmap
df_heatmap = df_heatmap.where(df_heatmap <0.90, 0)


plt.figure(figsize=(9, 6))
ax = sns.heatmap(df_heatmap, annot=True, cmap="viridis")

ax.set_xlabel("MF Clusters")
ax.set_ylabel("RCC CLusters")

# custom tick labels
#ax.set_xticklabels(["label1", "label2", "label3"], rotation=45, ha="right")
#ax.set_yticklabels(["row1", "row2", "row3"], rotation=0)

In [ ]:
#scores between all cell cell pairs + cluster assignments

new_df = pd.DataFrame()

for i in jaccard_df_compp["row_cluster"].sort_values().unique():
    for j in jaccard_df_compp["col_cluster"].sort_values().unique():
        new_df_tmp = jaccard_df_compp[(jaccard_df_compp["row_cluster"] == i) & (jaccard_df_compp["col_cluster"] == j)] 
        new_df_tmp = new_df_tmp.sort_values(by="weighted_jaccard", ascending=False).head(n=15)
        new_df = pd.concat([new_df, new_df_tmp])


for i, row in new_df.iterrows():
    value = df_heatmap.loc[row["row_cluster"], row["col_cluster"]]
    new_df.at[i, "median_jaccard"] = value

filtered  = new_df[(new_df["median_jaccard"] != 0)]

new_df.to_csv("all_jaccard_cellpairs_min_5_lr_interactions_all.csv", index=0)

In [ ]:
row_cluster_labels_df = pd.DataFrame(row_cluster_labels)
row_cluster_labels_df = pd.DataFrame(row_cluster_labels).rename(columns={0: "row_cluster"})
row_cluster_labels_df['RCC_cluster'] = row_cluster_labels_df.index


col_cluster_labels_df = pd.DataFrame(col_cluster_labels)
col_cluster_labels_df = pd.DataFrame(col_cluster_labels).rename(columns={0: "col_cluster"})
col_cluster_labels_df['MF_cluster'] = col_cluster_labels_df.index


row_cluster_labels_df[["RCC_Sender", "RCC_Receiver"]] = row_cluster_labels_df["RCC_cluster"].str.split("@", expand= True)
r_sender_counts = row_cluster_labels_df.groupby(["row_cluster", "RCC_Sender"]).size().unstack(fill_value=0)
r_receiver_counts = row_cluster_labels_df.groupby(["row_cluster", "RCC_Receiver"]).size().unstack(fill_value=0)
r_sender_counts2 = r_sender_counts.add_suffix("_as_sender")
r_receiver_counts2 = r_receiver_counts.add_suffix("_as_receiver")
row_combined_counts = pd.concat([r_sender_counts2, r_receiver_counts2], axis=1).fillna(0).astype(int)
row_combined_counts = row_combined_counts.T

col_cluster_labels_df[["MF_Sender", "MF_Receiver"]] = col_cluster_labels_df["MF_cluster"].str.split("@", expand= True)
c_sender_counts = col_cluster_labels_df.groupby(["col_cluster", "MF_Sender"]).size().unstack(fill_value=0)
c_receiver_counts = col_cluster_labels_df.groupby(["col_cluster", "MF_Receiver"]).size().unstack(fill_value=0)
c_sender_counts2 = c_sender_counts.add_suffix("_as_sender")
c_receiver_counts2 = c_receiver_counts.add_suffix("_as_receiver")
col_combined_counts = pd.concat([c_sender_counts2, c_receiver_counts2], axis=1).fillna(0).astype(int)
col_combined_counts = col_combined_counts.T

c_sender_counts = c_sender_counts.T
c_receiver_counts = c_receiver_counts.T

r_sender_counts = r_sender_counts.T
r_receiver_counts = r_receiver_counts.T

In [ ]:
#counts of cell types per cluster

lst = []

for i in row_combined_counts.columns:
    for j in col_combined_counts.columns:

        value = df_heatmap.loc[i, j]
        if value == 0:
                continue
        
        r_s = pd.DataFrame()
        r_r = pd.DataFrame()
        c_s = pd.DataFrame()
        c_r = pd.DataFrame()

        r_s["counts"] = r_sender_counts[i]
        r_s["percentage"] = (r_s["counts"]/(r_s["counts"].sum()))*100

        r_r["counts"] = r_receiver_counts[i]
        r_r["percentage"] = (r_r["counts"]/(r_r["counts"].sum()))*100

        c_s["counts"] = c_sender_counts[j]
        c_s["percentage"] = (c_s["counts"]/(c_s["counts"].sum()))*100

        c_r["counts"] = c_receiver_counts[j]
        c_r["percentage"] = (c_r["counts"]/(c_r["counts"].sum()))*100


        sorted_r_s = r_s.sort_values(by="counts", ascending=False).head(n=5)
        sorted_c_s = c_s.sort_values(by="counts", ascending=False).head(n=5)
        sorted_r_r = r_r.sort_values(by="counts", ascending=False).head(n=5)
        sorted_c_r = c_r.sort_values(by="counts", ascending=False).head(n=5)

        sorted_r_s = sorted_r_s[sorted_r_s["counts"]!=0]
        sorted_c_s = sorted_c_s[sorted_c_s["counts"]!=0]
        sorted_r_r = sorted_r_r[sorted_r_r["counts"]!=0]
        sorted_c_r = sorted_c_r[sorted_c_r["counts"]!=0]

        lst.append(f"Cluster Similarity {i} and {j}: {df_heatmap.loc[i, j]} \n Total number of cellpairs: {r_s["counts"].sum()} (RCC); {c_s["counts"].sum()} (MF) \n Unique Senders: {len(r_s.index[r_s["counts"] != 0].unique())} (RCC), {len(c_s.index[c_s["counts"] != 0].unique())} (MF) \n Unique Receivers: {len(r_r.index[r_r["counts"] != 0].unique())} (RCC), {len(c_r.index[c_r["counts"] != 0].unique())} (MF)")
        lst.append(sorted_r_s)
        lst.append(sorted_c_s)
        lst.append(sorted_r_r)
        lst.append(sorted_c_r)
        

lst
with open("top5_celltypes_per_cluster_dataset_comparison_normalized_LR_scores.txt", 'w') as f:
    for s in lst:
        f.write(str(s) + '\n\n\n')

In [ ]:
#RCC = RCC[RCC["lr_means_normed"] > 0.10]
#MF = MF[MF["lr_means_normed"] > 0.10]

mean_LR_RCC = RCC.groupby(["source", "target"])["lr_means_normed"].mean()
mean_LR_RCC.add_prefix("RCC_", axis=0)
mean_LR_MF = MF.groupby(["source", "target"])["lr_means_normed"].mean()

fig, ax = plt.subplots()
sns.violinplot(mean_LR_RCC, inner="quart", color=  "#FF944C")
print(np.quantile(mean_LR_RCC, 0.75))
print(np.quantile(mean_LR_MF, 0.75))
sns.violinplot(mean_LR_MF, inner="quart", color = "#94FF8F")
ax.set_ylabel("LR Means Normed")


mean_LR_RCC = pd.DataFrame(mean_LR_RCC).reset_index()
mean_LR_RCC = mean_LR_RCC.pivot(index = "source", columns = "target", values = "lr_means_normed")
mean_LR_RCC = mean_LR_RCC.sort_index(axis=0).sort_index(axis=1).add_prefix("RCC_", axis=0).add_prefix("RCC_", axis=1)

mean_LR_MF = pd.DataFrame(mean_LR_MF).reset_index()
mean_LR_MF = mean_LR_MF.pivot(index = "source", columns = "target", values = "lr_means_normed")
mean_LR_MF = mean_LR_MF.sort_index(axis=0).sort_index(axis=1).add_prefix("MF_", axis=0).add_prefix("MF_", axis=1)




In [ ]:
#sender to sender and receiver to receiver
jaccard_df_comp[["MF_sender", "MF_receiver"]] = jaccard_df_comp["MF_cluster"].str.split("@", expand=True)
jaccard_df_comp[["RCC_sender", "RCC_receiver"]] = jaccard_df_comp["RCC_cluster"].str.split("@", expand=True)

jaccard_df_comp_split_s = jaccard_df_comp.groupby(["MF_sender", "RCC_sender"])["weighted_jaccard"].mean()
jaccard_df_comp_split_r = jaccard_df_comp.groupby(["RCC_receiver", "MF_receiver"])["weighted_jaccard"].mean()

jaccard_df_comp_split_s = jaccard_df_comp_split_s.reset_index()
jaccard_df_comp_split_s = jaccard_df_comp_split_s.pivot(index = "RCC_sender", columns = "MF_sender", values= "weighted_jaccard").add_prefix("RCC_", axis=0).add_prefix("MF_", axis=1)

jaccard_df_comp_split_r = jaccard_df_comp_split_r.reset_index()
jaccard_df_comp_split_r = jaccard_df_comp_split_r.pivot(index = "MF_receiver", columns = "RCC_receiver", values= "weighted_jaccard").add_prefix("MF_", axis=0).add_prefix("RCC_", axis=1)
jaccard_df_comp_split = pd.concat((jaccard_df_comp_split_s, jaccard_df_comp_split_r))

print(np.quantile(jaccard_df_comp_split_s, 0.75))
print(np.quantile(jaccard_df_comp_split_r, 0.75))

In [ ]:
#sender and receiver probabilities

jaccard_df_comp_split_s_p = jaccard_df_comp_split_s.div(jaccard_df_comp_split_s.max(axis=1), axis=0) 
jaccard_df_comp_split_r_p = jaccard_df_comp_split_r.div(jaccard_df_comp_split_r.max(axis=1), axis=0) 


fig, ax = plt.subplots()
sns.violinplot(jaccard_df_comp_split_s_p.mean(), inner="quart", color=  "#FF944C")
sns.violinplot(jaccard_df_comp_split_r_p.mean(), inner="quart", color = "#94FF8F")

print(np.quantile(jaccard_df_comp_split_s_p, 0.75))
print(np.quantile(jaccard_df_comp_split_r_p, 0.75))

jaccard_df_comp_split_s_p = jaccard_df_comp_split_s_p[jaccard_df_comp_split_s_p >= 0.7]
jaccard_df_comp_split_r_p = jaccard_df_comp_split_r_p[jaccard_df_comp_split_r_p >= 0.62]

jaccard_df_comp_split_p = pd.concat((jaccard_df_comp_split_s_p, jaccard_df_comp_split_r_p))

In [ ]:
jaccard_df_comp_split_m = jaccard_df_comp_split_p
combined_matrices = pd.DataFrame()

median_LR_MF = median_LR_MF[median_LR_MF >=0.23958727957045906]
median_LR_RCC = median_LR_RCC[median_LR_RCC >=0.2265910763666835]

combined_matrices_LR = pd.concat((median_LR_MF, median_LR_RCC))

#thresholds
#combined_matrices_LR = combined_matrices_LR[combined_matrices_LR >= mean_quants]#0.23]
#jaccard_df_comp_split_m = jaccard_df_comp_split_m[jaccard_df_comp_split_m >= 0.65]

#setting Jaccard score to 1
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.mask(jaccard_df_comp_split_m < 0.05)
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.applymap(lambda x: 1 if pd.notna(x) else np.nan)

#normalizing Jaccard and LR before merging to be on same scale
#min_val = combined_matrices_LR.min()
#max_val = combined_matrices_LR.max()
#combined_matrices_LR = (combined_matrices_LR - min_val) / (max_val - min_val)
#min_val = jaccard_df_comp_split_m.min()
#max_val = jaccard_df_comp_split_m.max()
#jaccard_df_comp_split_m = (jaccard_df_comp_split_m - min_val) / (max_val - min_val)



combined_matrices = combined_matrices_LR.combine_first(jaccard_df_comp_split_m)
combined_matrices = combined_matrices.fillna(0)


g_comb = ig.Graph.Weighted_Adjacency(
    combined_matrices.to_numpy().tolist(), 
    mode= "directed",     
    attr="weight"
)

#node names
g_comb.vs["name"] = list(combined_matrices.index)
g_comb.vs["label"] = g_comb.vs["name"]
g_comb.vs["label_size"] = 10
degree = g_comb.degree()

clusters_comb = leidenalg.find_partition(g_comb, weights=g_comb.es["weight"], partition_type= leidenalg.CPMVertexPartition, resolution_parameter= 0.27,seed = 33)
print(clusters_comb)
#g_comb.vs["color"] = [clusters_comb.membership[i] for i in range(len(g_comb.vs))]

layout = g_comb.layout("fr") 



ig.plot(clusters_comb, layout = layout, vertex_size=30, edge_width=[w["weight"] for w in g_comb.es], edge_alpha= 0.4, target="clustered_igraph_quant.png", 
        bbox = (1500,1500), margin = 100)

#n = g_comb.to_networkx()

In [ ]:
#CPM Vertex  k per resolution plot

resolutions = np.linspace(0, 0.4, 100) 
results_res = []
results_k = []
results = []

runs = 100

for r in resolutions:
    aris = []
    nmi = []
    partition = leidenalg.find_partition(
        g_comb,
        leidenalg.CPMVertexPartition,
        weights=g_comb.es["weight"],
        resolution_parameter=r,
        seed = 33
    )
    modularity = partition.modularity
    results_res.append(r)
    results_k.append(len(partition))
    results.append((r, len(partition)))

#plt.bar(results_res, results_k, align="edge")
#plt.xticks(results_res)
#plt.yticks(results_k)
#plt.xlim(0,0.4)

resultsdf = pd.DataFrame(results)
ax = sns.lineplot(x = resultsdf[0], y = resultsdf[1])
ax.set(xlabel = "CPMVertex Resolutions", ylabel = "k clusters", yticks = results_k)
plt.show()

In [ ]:
#RBERVertex  k per resolution plot
resolutions = np.linspace(0.5, 3, 300) 
results_res = []
results_k = []
results = []

runs = 100

for r in resolutions:
    aris = []
    nmi = []
    partition = leidenalg.find_partition(
        g_comb,
        leidenalg.RBERVertexPartition,
        weights=g_comb.es["weight"],
        resolution_parameter=r,
        seed = 33
    )
    modularity = partition.modularity
    results_res.append(r)
    results_k.append(len(partition))
    results.append((r, len(partition)))


#plt.bar(results_res, results_k, align="edge")
#plt.xticks(results_res)
#plt.yticks(results_k)
#plt.xlim(0.5, 3)

resultsdf = pd.DataFrame(results)
ax = sns.lineplot(x = resultsdf[0], y = resultsdf[1])
ax.set(xlabel = "RBERVertex Resolutions", ylabel = "k clusters", yticks = results_k)
plt.show()

In [ ]:
#RBERVertex

jaccard_df_comp_split_m = jaccard_df_comp_split_p
combined_matrices = pd.DataFrame()


median_LR_MF = median_LR_MF[median_LR_MF >= 0.2399136639826615]
median_LR_RCC = median_LR_RCC[median_LR_RCC >= 0.2265910763666835]

combined_matrices_LR = pd.concat((median_LR_MF, median_LR_RCC))

#thresholds
#combined_matrices_LR = combined_matrices_LR[combined_matrices_LR >= 0.23]
#jaccard_df_comp_split_m = jaccard_df_comp_split_m[jaccard_df_comp_split_m >= 0.65]

#setting Jaccard score to 1
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.mask(jaccard_df_comp_split_m < 0.05)
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.applymap(lambda x: 1 if pd.notna(x) else np.nan)

#normalizing Jaccard and LR before merging to be on same scale
#min_val = combined_matrices_LR.min()
#max_val = combined_matrices_LR.max()
#combined_matrices_LR = (combined_matrices_LR - min_val) / (max_val - min_val)
#min_val = jaccard_df_comp_split_m.min()
#max_val = jaccard_df_comp_split_m.max()
#jaccard_df_comp_split_m = (jaccard_df_comp_split_m - min_val) / (max_val - min_val)



combined_matrices = combined_matrices_LR.combine_first(jaccard_df_comp_split_m)
combined_matrices = combined_matrices.fillna(0)


g_comb = ig.Graph.Weighted_Adjacency(
    combined_matrices.to_numpy().tolist(), 
    mode= "directed",     
    attr="weight"
)

#node names
g_comb.vs["name"] = list(combined_matrices.index)
g_comb.vs["label"] = g_comb.vs["name"]
g_comb.vs["label_size"] = 10
degree = g_comb.degree()

clusters_comb = leidenalg.find_partition(g_comb, weights=g_comb.es["weight"], partition_type= leidenalg.RBERVertexPartition, resolution_parameter= 1.15, seed = 33)
print(clusters_comb)
#g_comb.vs["color"] = [clusters_comb.membership[i] for i in range(len(g_comb.vs))]

layout = g_comb.layout("fr") 



ig.plot(clusters_comb, layout = layout, vertex_size=30, edge_width=[w["weight"] for w in g_comb.es], edge_alpha= 0.4, target="clustered_igraph_quant.png", 
        bbox = (1500,1500), margin = 100)

#n = g_comb.to_networkx()

In [ ]:
#RBConfigurationVertexPartition k per resolution plot
resolutions = np.linspace(0.5, 3, 300) 
results = []
results_res = []
results_k = []


runs = 100

for r in resolutions:
    aris = []
    nmi = []
    partition = leidenalg.find_partition(
        g_comb,
        leidenalg.RBConfigurationVertexPartition,
        weights=g_comb.es["weight"],
        resolution_parameter=r,
        seed = 33
    )
    modularity = partition.modularity
    results_res.append(r)
    results_k.append(len(partition))
    results.append((r, len(partition)))



#plt.bar(results_res, results_k, align="edge", color ="green")
#plt.xticks(results_res)
#plt.xticks(round(results_res[::35], 1))

#plt.yticks(results_k)
#plt.xlabel("Resolutions RBConfiguration")
#plt.ylabel("k clusters")
#plt.xlim(0.5, 3)

resultsdf = pd.DataFrame(results)
ax = sns.lineplot(x = resultsdf[0], y = resultsdf[1])
ax.set(xlabel = "RBConfiguration Resolutions", ylabel = "k clusters", yticks = results_k)
plt.show()

In [ ]:
#RBConfigurationVertexPartition

jaccard_df_comp_split_m = jaccard_df_comp_split_p
combined_matrices = pd.DataFrame()


mean_LR_MF = mean_LR_MF[mean_LR_MF >= 0.2399136639826615]
mean_LR_RCC = mean_LR_RCC[mean_LR_RCC >= 0.2265910763666835]

combined_matrices_LR = pd.concat((mean_LR_MF, mean_LR_RCC))

#thresholds
#combined_matrices_LR = combined_matrices_LR[combined_matrices_LR >= 0.23]
#jaccard_df_comp_split_m = jaccard_df_comp_split_m[jaccard_df_comp_split_m >= 0.65]

#setting Jaccard score to 1
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.mask(jaccard_df_comp_split_m < 0.05)
#jaccard_df_comp_split_m = jaccard_df_comp_split_m.applymap(lambda x: 1 if pd.notna(x) else np.nan)

#normalizing Jaccard and LR before merging to be on same scale
#min_val = combined_matrices_LR.min()
#max_val = combined_matrices_LR.max()
#combined_matrices_LR = (combined_matrices_LR - min_val) / (max_val - min_val)
#min_val = jaccard_df_comp_split_m.min()
#max_val = jaccard_df_comp_split_m.max()
#jaccard_df_comp_split_m = (jaccard_df_comp_split_m - min_val) / (max_val - min_val)



combined_matrices = combined_matrices_LR.combine_first(jaccard_df_comp_split_m)
combined_matrices = combined_matrices.fillna(0)


g_comb = ig.Graph.Weighted_Adjacency(
    combined_matrices.to_numpy().tolist(), 
    mode= "directed",     
    attr="weight"
)

#node names
g_comb.vs["name"] = list(combined_matrices.index)
g_comb.vs["label"] = g_comb.vs["name"]
g_comb.vs["label_size"] = 10
degree = g_comb.degree()

clusters_comb = leidenalg.find_partition(g_comb, weights=g_comb.es["weight"], partition_type= leidenalg.RBConfigurationVertexPartition, resolution_parameter= 1.25, seed = 33)
print(clusters_comb)
#g_comb.vs["color"] = [clusters_comb.membership[i] for i in range(len(g_comb.vs))]

layout = g_comb.layout("fr") 



ig.plot(clusters_comb, layout = layout, vertex_size=30, edge_width=[w["weight"] for w in g_comb.es], edge_alpha= 0.4, target="clustered_igraph_quant.png", 
        bbox = (1500,1500), margin = 100)

#n = g_comb.to_networkx()

In [ ]:
#silhoutte score clustering
labels = clusters_comb.membership


distances = []
for e in g_comb.es:
    w = e["weight"]
    dist = 1.0 / w if w > 0 else 1e9
    distances.append(dist)
g_comb.es["dist"] = distances


D = np.array(g_comb.distances(weights="dist"))


sil = silhouette_score(D, labels, metric="precomputed")
print("Silhouette score:", sil)


In [ ]:
#RBConfig res 1.25 after removing glom endo
cluster_dict = {"Monocytes" : ["MF_CD16_monocytes", "MF_Classical_monocytes", "MF_EBM_prog", "MF_GMPs",
    "MF_Inflammatory_CD14_monocytes", "MF_Late_Neutro_Prog", "MF_Macrophages",
    "MF_Neutrophils", "MF_SPP1_Macrophages", "MF_cDC1", "MF_cDC2", "RCC_Classical monocytes", "RCC_Non-classical monocytes", "RCC_TAM 1", "RCC_TAM 2", "RCC_TAM 3", "RCC_TAM 4"],
"Lymphocytes" : ["MF_B_cells", "MF_CD8_T_cells","MF_NK_T_cells", "RCC_B cells",
    "RCC_CD8 T cells", "RCC_Cytotoxic T cells", "RCC_IGHG-high plasma cells",
    "RCC_NK cells", "RCC_Plasma cells", "RCC_Regulatory T cells",
    "RCC_Resting/memory T cells"],#, "RCC_Proximal tubule"],
"Tumor_prog" : ["MF_B_cell_prog", "MF_Plasma_cells",
                 "MF_Early_Neutro_Prog", "MF_HSPCs", "MF_MEPs", "MF_MK_prog",
    "RCC_DCT/CNT", "RCC_Epithelial progenitor-like cells", "RCC_Principal cells",
    "RCC_Tumor cells 1", "RCC_Tumor cells 2", "RCC_Tumor cells 3", "RCC_tAL of LOH"],
"Endothelial" : [ "MF_pDCs", "MF_Arterial_EC", "MF_Erythroid_cells", "MF_Late_Erythroid_Prog",
    "MF_Sinusoidal_EC", "RCC_AVR", "RCC_DVR", "RCC_Tumor AVR-like vasculature",
    "RCC_Tumor vasculature 1", "RCC_Tumor vasculature 2", "RCC_Tumor vasculature 3", "RCC_Tumor vasculature 4"],
"Stromal" : ["MF_Adipo_CAR", "MF_Mature_MKs", "MF_OLCs", "MF_Osteoblasts", "RCC_Mesangial/vSMCs",
    "RCC_Myofibroblasts", "RCC_vSMCs"],
"Mast_cells" : ["MF_Mast_cells", "RCC_Mast cells"]}

In [ ]:
#plotting subgraphs
subgraphs = [g_comb.subgraph(cluster) for cluster in clusters_comb]

fig, axes = plt.subplots(1, len(subgraphs), figsize=(10*len(subgraphs), 10))
import matplotlib.cm as cm
from matplotlib import colors as mcolors

cluster_names = list(cluster_dict.keys())
cluster_names = ["Myeloids", "Tumor/prog", "Endothelial", "Lymphocytes", "Stromal", "Mast cells"]

# pick a colormap
cmap = cm.get_cmap("tab10", len(subgraphs))  # up to 10 distinct colors

colors = [cmap(i) for i in range(len(subgraphs))]
edge_alpha = 0.4  # edge transparency
edge_color = mcolors.to_rgba("black", alpha=edge_alpha)


for i, (sg, cluster_name) in enumerate(zip(subgraphs, cluster_names)):
    filename = f"{cluster_name.replace('/', '_').replace(' ', '_')}_igraph.png"
    layout = sg.layout("fr")  # Fruchterman-Reingold (force-directed)
    color = colors[i]
    ig.plot(
        sg,
        #target=axes[i],
        layout=layout,
        vertex_size=30,
        vertex_label=sg.vs["name"],
        vertex_label_size = 15,
        vertex_label_dist = 2,
        target=filename,
        edge_width=[w for w in sg.es["weight"]],
        vertex_color=[color] * sg.vcount(),
        edge_color=[edge_color] * sg.ecount(),
        margin=20
    )
    axes[i].set_title(f"{cluster_name}")

plt.tight_layout()
plt.show()

#for clust in clusters_comb:
#    ig.plot(clust, layout = layout, vertex_size=30, edge_width=[w["weight"] for w in g_comb.es], edge_alpha= 0.4, target="clustered_igraph_quant.png", 
#        bbox = (1500,1500), margin = 100)

In [ ]:
#saving subpllots formatted (plotting subgraphs v2)

fig, axes = plt.subplots(1, len(subgraphs), figsize=(12*len(subgraphs), 12))

cluster_names = ["Myeloids", "Tumor/prog", "Endothelial", "Lymphocytes", "Stromal", "Mast cells"]


cmap = cm.get_cmap("tab10", len(subgraphs))
colors = [cmap(i) for i in range(len(subgraphs))]


edge_alpha = 0.4
edge_color = mcolors.to_rgba("black", alpha=edge_alpha)

def vertex_color_by_prefix(name):
    if name.startswith("RCC"):
        return "#FF944C"  
    elif name.startswith("MF"):
        return "#7FCE7B"
    else:
        return "#CCCCCC"  # 

legend_elements = [
    Patch(facecolor="#FF944C", edgecolor="black", label="RCC"),
    Patch(facecolor="#7FCE7B", edgecolor="black", label="MF")
]

for i, (sg, cluster_name) in enumerate(zip(subgraphs, cluster_names)):
    layout = sg.layout("fr")  


    vertex_colors = [vertex_color_by_prefix(v["name"]) for v in sg.vs]
    filename = f"{cluster_name.replace('/', '_').replace(' ', '_')}_igraph.png"


    ig.plot(
        sg,
        target=filename,
        layout=layout,
        vertex_size=20,
        vertex_label=sg.vs["name"],
        vertex_label_size=15,
        vertex_label_dist=2,
        edge_width=sg.es["weight"],
        vertex_color=vertex_colors,
        edge_color=[edge_color] * sg.ecount(),
        margin=80
    )


    img = plt.imread(filename)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img)
    ax.axis("off")
   
    ax.legend(
        handles=legend_elements,
        loc="upper left",
        frameon=False,
        fontsize=12,
        ncol=1
    )

    outname = f"{cluster_name.replace('/', '_').replace(' ', '_')}_igraph_with_legend.png"
    plt.savefig(outname, bbox_inches="tight", dpi=300)
    plt.close(fig)


In [ ]:
# pyvoi bc i had import issues (chatgpt version delivers same results)
import numpy as np

import numpy.typing as npt
import warnings


def VI_np(labels1,labels2,return_split_merge=False):
    assert len(labels2)==len(labels1)
    size=len(labels2)

    mutual_labels=(labels1.astype(np.uint64)<<32)+labels2.astype(np.uint64)

    sm_unique,sm_inverse,sm_counts=np.unique(labels2,return_inverse=True,return_counts=True)
    fm_unique,fm_inverse,fm_counts=np.unique(labels1,return_inverse=True,return_counts=True)
    _,mutual_inverse,mutual_counts=np.unique(mutual_labels,return_inverse=True,return_counts=True)

    terms_mutual = -np.log(mutual_counts/size)*mutual_counts/size
    terms_mutual_per_count=terms_mutual[mutual_inverse]/mutual_counts[mutual_inverse]
    terms_sm = -np.log(sm_counts/size)*sm_counts/size
    terms_fm = -np.log(fm_counts/size)*fm_counts/size
    if not return_split_merge:
        terms_mutual_sum=np.sum(terms_mutual_per_count)
        vi_split=terms_mutual_sum-terms_sm.sum()
        vi_merge=terms_mutual_sum-terms_fm.sum()
        vi=vi_split+vi_merge
        return vi,vi_split,vi_merge

    vi_split_each=np.zeros(len(sm_unique))
    np.add.at(vi_split_each,sm_inverse,terms_mutual_per_count)
    vi_split_each-=terms_sm
    vi_merge_each=np.zeros(len(fm_unique))
    np.add.at(vi_merge_each,fm_inverse,terms_mutual_per_count)
    vi_merge_each-=terms_fm

    vi_split=np.sum(vi_split_each)
    vi_merge=np.sum(vi_merge_each)
    vi=vi_split+vi_merge

    i_splitters=np.argsort(vi_split_each)[::-1]
    i_mergers=np.argsort(vi_merge_each)[::-1]

    vi_split_sorted=vi_split_each[i_splitters]
    vi_merge_sorted=vi_merge_each[i_mergers]

    splitters=np.stack([vi_split_sorted,sm_unique[i_splitters]],axis=1)
    mergers=np.stack([vi_merge_sorted,fm_unique[i_mergers]],axis=1)
    return vi,vi_split,vi_merge,splitters,mergers


In [ ]:
#scores validating that clustering stayed consistent
resolutions = np.linspace(0.15, 0.35, 20) 
results = []


def get_partition_labels(g, resolution, seed):
    partition = leidenalg.find_partition(
        g_comb,
        leidenalg.CPMVertexPartition,
        resolution_parameter=resolution,
        weights=g_comb.es["weight"],
        seed=seed+500
    )
    return partition.membership

runs = 100

for r in resolutions:
    aris = []
    nmi = []
    partition = leidenalg.find_partition(
        g_comb,
        leidenalg.CPMVertexPartition,
        weights=g_comb.es["weight"],
        resolution_parameter=r
    )
    modularity = partition.modularity

    labels = [get_partition_labels(g_comb, r, seed=i) for i in range(runs)]

    for i in range(runs):
        for j in range(i+1, runs):
            aris.append(adjusted_rand_score(labels[i], labels[j]))
            nmi.append(normalized_mutual_info_score(labels[i], labels[j]))
            vi,vi_split,vi_merge=VI_np(np.array(labels[i]), np.array(labels[j]),return_split_merge=False)
    results.append((r, np.mean(aris), np.mean(nmi), np.mean(vi), modularity, len(partition)))

for r, aris_mean, nmi, vi, mod, k in results:
    print(f"Resolution {r:.2f}: mean ARI = {aris_mean:.3f}; mean NMI = {nmi:.3f}; mean VI = {vi:.3f}; modularity={mod:.3f}; n_clusters={k}")

In [ ]:
#full feature matrix

#RCC = pd.read_csv("/home/larissa/Documents/Masterarbeit/RCC_results/liana_log_norm/no_subsets/T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("/home/larissa/Documents/Masterarbeit/MF_results/liana/all_celltypes/MF_lr_ready.csv", index_col=0)

#RCC = pd.read_csv("D:\\studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subsets\\T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("D:\\studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col=0)

RCC = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col = 0)
MF = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col = 0)

RCC["lr_pair"] = RCC["gene_A"] + "_" + RCC["gene_B"]
MF["lr_pair"] = MF["gene_A"] + "_" + MF["gene_B"]

RCC["cluster_pair"] = RCC["source"] + "@" + RCC["target"]
MF["cluster_pair"] = MF["source"] + "@" + MF["target"]

#lr_threshold = 0.0
RCC_filtered = RCC[(RCC["lr_means"] >= 0.0) | (RCC["lr_means"] <=  -abs(0.0))]
MF_filtered = MF[(MF["lr_means"] >= 0.0) | (MF["lr_means"] <= -abs(0.0))]

RCC_grouped = (RCC_filtered.groupby("cluster_pair")
               .apply(lambda df: dict(zip(df["lr_pair"], df["lr_means"])))
               .to_dict())

MF_grouped = (MF_filtered.groupby("cluster_pair")
              .apply(lambda df: dict(zip(df["lr_pair"], df["lr_means"])))
              .to_dict())

def weighted_jaccard_dict(dict1, dict2):
    all_keys = set(dict1.keys()).union(dict2.keys())
    vec1 = np.array([dict1.get(i, 0.0) for i in all_keys])
    vec2 = np.array([dict2.get(i, 0.0) for i in all_keys])
    return np.minimum(vec1, vec2).sum() / np.maximum(vec1, vec2).sum() if np.maximum(vec1, vec2).sum() != 0 else 0.0

# pairwise similarity
results = []
for MF_cluster, MF_dict in MF_grouped.items():
    for RCC_cluster, RCC_dict in RCC_grouped.items():
        score = weighted_jaccard_dict(MF_dict, RCC_dict)
        results.append((MF_cluster, RCC_cluster, score))
        results.append((RCC_cluster, MF_cluster, score))

for MF_cluster, MF_dict in MF_grouped.items():
    for MF_cluster2, MF_dict2 in MF_grouped.items():
        score = weighted_jaccard_dict(MF_dict, MF_dict2)
        results.append((MF_cluster, MF_cluster2, score))

for RCC_cluster, RCC_dict in RCC_grouped.items():
    for RCC_cluster2, RCC_dict2 in RCC_grouped.items():
        score = weighted_jaccard_dict(RCC_dict, RCC_dict2)
        results.append((RCC_cluster, RCC_cluster2, score))



jaccard_df = pd.DataFrame(results, columns=["cluster_pair_1", "cluster_pair_2", "weighted_jaccard"])
#jaccard_df.to_csv("jaccard_score_weighted_cluster_panirs_thresholded.csv", index=False)

In [ ]:
#full feature heatmap

import seaborn as sns
import pandas as pd
import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage

#same clusters are not 1 but do they need to be for visualization? no

# pivot to square matrix

sim_matrix = jaccard_df.pivot(
    index="cluster_pair_1", columns="cluster_pair_2", values="weighted_jaccard"
)

dist_matrix = 1 - sim_matrix

#flatten to condensed distance matrix
condensed_dist = squareform(dist_matrix.values, checks=False)

# hierarchical clustering 
linkage_matrix = linkage(condensed_dist, method='average')

clust_map = sns.clustermap(sim_matrix, row_linkage=linkage_matrix, col_linkage=linkage_matrix, row_cluster=True, col_cluster=True, figsize=(30,30), cmap="viridis")
#sim_matrix.to_csv("sim_matrix_jaccard.csv")
clust_map.savefig("jaccard_score_dist_linkage_full_feature_average_linkage.png") 

In [ ]:
#full feature without axis ticks 

#
sim_matrix = jaccard_df.pivot(
    index="cluster_pair_1", columns="cluster_pair_2", values="weighted_jaccard"
)

dist_matrix = 1 - sim_matrix

condensed_dist = squareform(dist_matrix.values, checks=False)

linkage_matrix = linkage(condensed_dist, method='average')

clust_map = sns.clustermap(
    sim_matrix,
    row_linkage=linkage_matrix,
    col_linkage=linkage_matrix,
    row_cluster=True,
    col_cluster=True,
    figsize=(30, 30),
    cmap="viridis",
    cbar_kws={"shrink": 0.4}


clust_map.ax_heatmap.set_xticks([])
clust_map.ax_heatmap.set_yticks([])


clust_map.ax_row_dendrogram.set_ylabel("Clusters", fontsize=18)
clust_map.ax_col_dendrogram.set_xlabel("Clusters", fontsize=18)


clust_map.cax.tick_params(labelsize=18)

clust_map.savefig("jaccard_score_dist_linkage_full_feature_average_linkage.png")
plt.close()


In [ ]:
import prince

dist_matrix_rect = 1 - heatrmap_df

ca = prince.CA(n_components=2)
ca = ca.fit(dist_matrix)

row_coords = ca.row_coordinates(dist_matrix)
col_coords = ca.column_coordinates(dist_matrix)


plt.figure(figsize=(10, 10))
plt.scatter(row_coords[0], row_coords[1], c='red', label='Rows')
plt.scatter(col_coords[0], col_coords[1], c='blue', label='Columns')

#labels
#or i, txt in enumerate(dist_matrix_rect.index):
 #   plt.annotate(txt, (row_coords[0][i], row_coords[1][i]), color='red')

#for i, txt in enumerate(dist_matrix_rect.columns):#
 #   plt.annotate(txt, (col_coords[0][i], col_coord#s[1][i]), color='blue')


plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#NOT full feature (sparse matrix)

#same clusters are not 1 but do they need to be for visualization? no

mat1 = jaccard_df_comp.pivot(
    index="RCC_cluster", columns="MF_cluster", values="weighted_jaccard"
)
mat2 = jaccard_df_comp.pivot(
    index="MF_cluster", columns="RCC_cluster", values="weighted_jaccard"
)

mat3 = pd.concat([mat1, mat2])
#mat3 = mat3.sort_index(axis=0).sort_index(axis=1)
mat3 = mat3.fillna(0)
dist_matrix = 1 - mat3



linkage_matrix = linkage(condensed_dist, method='average')

clust_map = sns.clustermap(mat3, row_linkage=linkage_matrix, col_linkage=linkage_matrix, row_cluster=True, col_cluster=True, figsize=(100,100), cmap="viridis")
#sim_matrix.to_csv("sim_matrix_jaccard.csv")
clust_map.ax_heatmap.set_xticks([])
clust_map.ax_heatmap.set_yticks([])

clust_map.ax_row_dendrogram.set_ylabel("Clusters", fontsize=18)
clust_map.ax_col_dendrogram.set_xlabel("Clusters", fontsize=18)


clust_map.cax.tick_params(labelsize=18)

clust_map.savefig("jaccard_score_dist_linkage_sparse_mat_average_linkage.png") 